In [148]:
from bs4 import BeautifulSoup
import requests

import unicodedata
from csv import writer
import re

import pandas as pd
import numpy as np

import json
from pyspark.sql import SparkSession
import os

#folium
import geopandas as gpd
import re
import folium

In [149]:

output_dir = '../data/raw/'


In [150]:
'''


property and elector count by postcode


'''

#url: https://discover.data.vic.gov.au/dataset/victorian-electors-by-locality-postcode-and-electorates 


columns = ['Locality Name','Post_x000D_\nCode','Property_x000D_\nCount','Elector_x000D_\nCount']
LocalityFinder_postcode = pd.read_excel('LocalityFinder_postcode.xlsx',
                                        sheet_name= 'Place_Names_Electronic',
                                        header=2)
LocalityFinder_postcode= LocalityFinder_postcode[columns].rename({'Post_x000D_\nCode': 'postcode',
                                                             'Locality Name':'suburb_name',
                                                             'Property_x000D_\nCount': 'property_count',
                                                            'Elector_x000D_\nCount':'elector_count'}, axis='columns')
property_and_elector = LocalityFinder_postcode.groupby('postcode').sum()

#output
filename = 'property_and_elector_by_postcode.csv'
output_dir_full = f'{output_dir}{filename}'
property_and_elector.to_csv(output_dir_full)

,property_count,elector_count
postcode,,
3000,15408,9168
3002,3308,3823
3003,3068,3265
3004,6439,6387
3006,10636,8631
...,...,...
3990,53,99
3991,199,336
3992,575,863


In [ ]:
"""
Download FOI zip file
1. open url:https://datashare.maps.vic.gov.au/search?md=019d7631-1234-5112-9f21-8f7346647b61
2. add to cart and check out with email
3. open url in email recieved, download zip file
4. find folder with name 'VMFEAT'
5. move folder under raw data folder


"""

In [201]:
#PTV
ptv_sf = gpd.read_file("../data/raw/PTV/PTV_METRO_TRAIN_STATION.shp")
ptv_sf.to_csv('staiton.csv')

In [198]:
# sf stands for shape file
sf = gpd.read_file("../data/raw/VMFEAT/FOI_POINT.shp")
VIC_FOI = sf.loc[sf['STATE']=='VIC']

In [161]:
'''


hospital


'''
comunity_sector = ['communication service','NAME','geometry']
hospital = VIC_FOI.loc[VIC_FOI['FTYPE'] == 'hospital']
hospital_df = hospital[hospital_columns]
hospital_df = hospital_df.dropna(subset=['FEATSUBTYP','NAME','geometry'])
hospital_df

,FEATSUBTYP,NAME,geometry
14522,general hospital,THOMAS EMBLING HOSPITAL,MULTIPOINT (145.01273 -37.78884)
15261,general hospital (emergency),BOX HILL HOSPITAL,MULTIPOINT (145.11837 -37.81390)
15307,general hospital (emergency),DANDENONG HOSPITAL,MULTIPOINT (145.21851 -37.97699)
15335,general hospital (emergency),MAROONDAH HOSPITAL,MULTIPOINT (145.25541 -37.80699)
17402,general hospital,NORTHERN COMMUNITY CARE UNIT,MULTIPOINT (145.02150 -37.73600)
...,...,...,...
35898,general hospital,WYNDHAM CLINIC,MULTIPOINT (144.70076 -37.88702)
35899,day procedure centre,WESTERN RADIATION ONCOLOGY CENTRE,MULTIPOINT (144.88566 -37.79306)
35901,general hospital,URSULA FRAYNE CENTRE,MULTIPOINT (144.88698 -37.79161)
35902,general hospital,THE BAYS HOSPITAL MORNINGTON,MULTIPOINT (145.04137 -38.22296)


In [162]:
#output
filename = 'hospital.csv'
output_dir_full = f'{output_dir}{filename}'
hospital_df.to_csv(output_dir_full)

In [163]:
'''

emergency services: police station, amubulance


'''
#filter out important emergency sevice
target_emergency_service= ['police station',
                           'ambulance station',
                           'fire station',
                           'fire station (forest industry)']
selected_columns = ['NAME_LABEL','FEATSUBTYP','geometry']

emergency_service_df = VIC_FOI.loc[VIC_FOI['FTYPE'] == 'emergency facility'][selected_columns]
emergency_service_df = emergency_service_df.loc[emergency_service_df['FEATSUBTYP']\
                                                .isin(target_emergency_service)]
emergency_service_df = emergency_service_df.dropna(subset=['NAME_LABEL','geometry'])
emergency_service_df

,NAME_LABEL,FEATSUBTYP,geometry
2244,Alexandra Police Station,police station,MULTIPOINT (145.70778 -37.18862)
2943,Anglesea Police Station,police station,MULTIPOINT (144.19223 -38.40339)
2996,Kyneton Ambulance Station,ambulance station,MULTIPOINT (144.46919 -37.25439)
2998,Landsborough Police Station,police station,MULTIPOINT (143.12854 -37.00363)
2999,Lexton Police Station,police station,MULTIPOINT (143.51382 -37.27525)
...,...,...,...
42616,Clayton Fire Station,fire station,MULTIPOINT (145.11665 -37.93900)
42636,Croydon Fire Station,fire station,MULTIPOINT (145.28942 -37.79940)
42639,Deer Park Fire Station,fire station,MULTIPOINT (144.77839 -37.77039)
42646,Eastern Hill Fire Station,fire station,MULTIPOINT (144.97545 -37.80889)


In [164]:
#output
filename = 'emergency_service.csv'
output_dir_full = f'{output_dir}{filename}'
emergency_service_df.to_csv(output_dir_full)



In [167]:
'''


public service: 
swimming pool,libary museum,art gallery


'''
service_type= ['sport facility',
               'cultural centre',
               'community space']

public_service = VIC_FOI.loc[VIC_FOI['FTYPE']\
                             .isin(service_type)]
target_service = ['camp ground',
                  'swimming pool',
                  'library',
                  'museum',
                  'art gallery',
                  'aquarium',
                  'observatory']
selected_columns = ['FTYPE','FEATSUBTYP','NAME','geometry'] 
public_service = public_service.loc[public_service['FEATSUBTYP']\
                             .isin(target_service)][selected_columns]
public_service_df = public_service.dropna(subset=['NAME','geometry'])
public_service_df

,FTYPE,FEATSUBTYP,NAME,geometry
640,community space,camp ground,BOYS CAMPING AREA,MULTIPOINT (145.85977 -37.31971)
656,community space,camp ground,GLENLUCE SPRINGS CAMPING GROUND,MULTIPOINT (144.23079 -37.16410)
664,community space,camp ground,KENDALS CAMPING AREA,MULTIPOINT (145.85116 -37.30975)
667,community space,camp ground,OLD RUBICON TOWN CAMPSITE,MULTIPOINT (145.86213 -37.32509)
842,community space,camp ground,HAPPY VALLEY CROSSING CAMPGROUND,MULTIPOINT (143.57964 -37.75121)
...,...,...,...,...
48429,community space,camp ground,TOBACCO FLAT CAMPING AREA,MULTIPOINT (146.31503 -37.21701)
48443,community space,camp ground,LAKE BOORT CAMPING AREA,MULTIPOINT (143.73205 -36.12462)
48446,community space,camp ground,PIGS POINT STREAMSIDE RESERVE,MULTIPOINT (147.23738 -36.27991)
48447,community space,camp ground,POSTPATCH,MULTIPOINT (146.19279 -36.09099)


In [168]:
#output
filename = 'public_service.csv'
output_dir_full = f'{output_dir}{filename}'
public_service_df.to_csv(output_dir_full)



In [172]:
'''


care facility (child,disability,aged)



'''

selected_columns = ['NAME_LABEL','FEATSUBTYP','geometry']
care_facility_df = VIC_FOI.loc[VIC_FOI['FTYPE'] =='care facility'][selected_columns]
care_facility_df['geometry']

2992     MULTIPOINT (147.08039 -37.96556)
10296    MULTIPOINT (147.06835 -38.10475)
10568    MULTIPOINT (141.58265 -38.34137)
10678    MULTIPOINT (141.66050 -36.32953)
11639    MULTIPOINT (145.01799 -37.70365)
                       ...               
48124    MULTIPOINT (144.97040 -37.74139)
48131    MULTIPOINT (144.34140 -38.30323)
48232    MULTIPOINT (147.64152 -37.80822)
48245    MULTIPOINT (145.17646 -37.84408)
48318    MULTIPOINT (144.70581 -37.62307)
Name: geometry, Length: 5780, dtype: geometry

In [171]:
#output
filename = 'care_facility.csv'
output_dir_full = f'{output_dir}{filename}'
care_facility_df.to_csv(output_dir_full)

